In [16]:
import pandas as pd
import numpy as np
from music21 import converter, midi, interval, pitch
from mido import MidiFile
import miditoolkit
import os
from os import walk
import json
from tokenizing_functions import extract_events, get_file_and_dirnames
#from helper_functions import get_file_and_dirnames
#from analysis_functions import analyse_data_folder
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

PATH_TRANSPOSED = "../0_data/4_preprocessed_sets"

In [3]:
MIN_DURATION_DENOMINATOR = 32
DURATION_STEPS = 64
POSITION_STEPS = 16
TICKS_PER_BEAT = 1024
TRIOLE_POS_1 = (TICKS_PER_BEAT/12).__round__()
TRIOLE_POS_2 = (TICKS_PER_BEAT/6).__round__()
TICKS_PER_MIN_DURATION = TICKS_PER_BEAT*4/MIN_DURATION_DENOMINATOR
DURATION_BINS = np.arange(TICKS_PER_MIN_DURATION, (TICKS_PER_MIN_DURATION*DURATION_STEPS)+1, TICKS_PER_MIN_DURATION, dtype=int)

start_position_tokens = 37 + DURATION_STEPS + 1
end_position_tokens = start_position_tokens + POSITION_STEPS

word2message = {0: "Bar_None"}
for i in range(1, 37):
    word2message[i] = f"Note-On_{i+59}"
for i in range(37, start_position_tokens-1):
    word2message[i] = f"Note-Duration_{i-36}"
word2message[start_position_tokens-1] = "Note-Duration_triole"
for i in range(start_position_tokens, end_position_tokens):
    word2message[i] = f"Position_{i-start_position_tokens+1}/{POSITION_STEPS}"
word2message[end_position_tokens] = "Position-Triole_1"
word2message[end_position_tokens+1] = "Position-Triole_2"

message2word = {v: k for k, v in word2message.items()}
message2word

{'Bar_None': 0,
 'Note-On_60': 1,
 'Note-On_61': 2,
 'Note-On_62': 3,
 'Note-On_63': 4,
 'Note-On_64': 5,
 'Note-On_65': 6,
 'Note-On_66': 7,
 'Note-On_67': 8,
 'Note-On_68': 9,
 'Note-On_69': 10,
 'Note-On_70': 11,
 'Note-On_71': 12,
 'Note-On_72': 13,
 'Note-On_73': 14,
 'Note-On_74': 15,
 'Note-On_75': 16,
 'Note-On_76': 17,
 'Note-On_77': 18,
 'Note-On_78': 19,
 'Note-On_79': 20,
 'Note-On_80': 21,
 'Note-On_81': 22,
 'Note-On_82': 23,
 'Note-On_83': 24,
 'Note-On_84': 25,
 'Note-On_85': 26,
 'Note-On_86': 27,
 'Note-On_87': 28,
 'Note-On_88': 29,
 'Note-On_89': 30,
 'Note-On_90': 31,
 'Note-On_91': 32,
 'Note-On_92': 33,
 'Note-On_93': 34,
 'Note-On_94': 35,
 'Note-On_95': 36,
 'Note-Duration_1': 37,
 'Note-Duration_2': 38,
 'Note-Duration_3': 39,
 'Note-Duration_4': 40,
 'Note-Duration_5': 41,
 'Note-Duration_6': 42,
 'Note-Duration_7': 43,
 'Note-Duration_8': 44,
 'Note-Duration_9': 45,
 'Note-Duration_10': 46,
 'Note-Duration_11': 47,
 'Note-Duration_12': 48,
 'Note-Duration_13

In [4]:
dir = "17_POP909-Dataset-master"
file = "111.mid"
files,_ = get_file_and_dirnames(f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}')
files.sort()
files[:10]

['002.mid',
 '003.mid',
 '004.mid',
 '005.mid',
 '006.mid',
 '007.mid',
 '008.mid',
 '009.mid',
 '010.mid',
 '011.mid']

In [5]:
path = f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}/100.mid'
"""note_items = convert_to_note_items(path)
note_items_shifts = compute_shifts(note_items)
max_time = note_items_shifts[-1]["end"]
grouped_items = group_items(note_items_shifts, max_time)
events = item2event(grouped_items)
events"""

'note_items = convert_to_note_items(path)\nnote_items_shifts = compute_shifts(note_items)\nmax_time = note_items_shifts[-1]["end"]\ngrouped_items = group_items(note_items_shifts, max_time)\nevents = item2event(grouped_items)\nevents'

In [9]:
events = extract_events(path)
tokens = [message2word[f"{e['name']}_{e['value']}"] for e in events]
tokens[:10]


[0, 112, 17, 38, 114, 17, 38, 116, 17, 38]

In [11]:
token_dict = {}
for file in tqdm(files):
    path = f'{PATH_TRANSPOSED}/c)_transposed_octave/{dir}/{file}'
    events = extract_events(path)
    tokens = [message2word[f"{e['name']}_{e['value']}"] for e in events]
    token_dict[file] = tokens


100%|██████████| 803/803 [00:21<00:00, 37.27it/s]


In [15]:
len(token_dict.keys())

803

In [17]:
with open('data.json', 'w') as fp:
    json.dump(token_dict, fp)

In [138]:
def word_to_event(words, word2message):
    events = []
    for word in words:
        event_name, event_value = word2message.get(word).split('_')
        events.append({
            "name": event_name,
            "time": None,
            "value": event_value,
            "text": None
        })
    return events

def write_midi(words, word2event, output_path, prompt_path=None):
    events = word_to_event(words, word2event)
    # get downbeat and note (no time)
    temp_notes = []
    for i in range(len(events)-3):
        if events[i]["name"] == 'Bar' and i > 0:
            temp_notes.append('Bar')
        elif events[i]["name"] == 'Position':
            # get position bin 
            position = int(events[i]["value"].split('/')[0]) - 1
            # get triole position
            if events[i+1]["name"] == 'Position-Triole':
                triole_position = int(events[i+1]["value"])
                n = 1
            else:
                triole_position = 0
                n = 0
            if events[i+n+1]["name"] == 'Note-On' and \
            events[i+n+2]["name"] == 'Note-Duration':
                # pitch
                pitch = int(events[i+n+1]["value"])
            # duration
            if events[i+n+3]["name"] != 'Note-Duration':
                index = int(events[i+n+2]["value"])-1
                duration = DURATION_BINS[index]
            elif events[i+n+3]["value"] == 'triole':
                index = int(events[i+n+2]["value"])-1
                duration = int(DURATION_BINS[index] / 3)
            # adding
            temp_notes.append([position, triole_position ,pitch, duration])
    # get specific time for notes
    ticks_per_bar = TICKS_PER_BEAT * 4 # assume 4/4
    notes = []
    current_bar = 0
    for note in temp_notes:
        if note == 'Bar':
            current_bar += 1
        else:
            position, triole_position, pitch, duration = note
            # position (start time)
            current_bar_st = current_bar * ticks_per_bar
            current_bar_et = (current_bar + 1) * ticks_per_bar
            flags = np.linspace(current_bar_st, current_bar_et, POSITION_STEPS, endpoint=False, dtype=int)
            if triole_position == 0:
                st = flags[position]
            elif triole_position == 1:
                st = flags[position] + TRIOLE_POS_1
            elif triole_position == 2:
                st = flags[position] + TRIOLE_POS_2
            # duration (end time)
            et = st + duration
            notes.append(miditoolkit.Note(100, pitch, st, et))
    # write
    if prompt_path:
        midi = miditoolkit.midi.parser.MidiFile(prompt_path)
        last_time = TICKS_PER_BEAT * 4 * 4
        # note shift
        for note in notes:
            note.start += last_time
            note.end += last_time
        midi.instruments[0].notes.extend(notes)
    else:
        midi = miditoolkit.midi.parser.MidiFile()
        midi.ticks_per_beat = TICKS_PER_BEAT
        # write instrument
        inst = miditoolkit.midi.containers.Instrument(0, is_drum=False)
        inst.notes = notes
        midi.instruments.append(inst)
        

    # write
    midi.dump(output_path)
    print("midi saved")

In [139]:
PATH_TEST = "../0_data/99_test"

# when paths not exist, create directories
if not os.path.exists(PATH_TEST):
    os.makedirs(PATH_TEST)

In [140]:
write_midi(tokens, word2message, f"{PATH_TEST}/test.mid")

midi saved
